# Step using amortized VBD and visualize

In [1]:
import os
import torch
import mediapy

from pathlib import Path
working_dir = Path.cwd()
while working_dir.name != 'gpudrive':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir.parent)

from gpudrive.env.config import EnvConfig, RenderConfig
from gpudrive.env.dataset import SceneDataLoader
from gpudrive.env.env_torch import GPUDriveTorchEnv
from gpudrive.datatypes.trajectory import VBDTrajectory
from gpudrive.visualize.utils import img_from_fig

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

In [2]:
#Init GPUDrive env
device = "cpu"

env_config = EnvConfig(
    dynamics_model="state", # Use state-based dynamics model
    dist_to_goal_threshold=1e-5, # Trick to make sure the agents don't disappear when they reach the goal
    init_mode = 'all_non_trivial',
    max_controlled_agents=32,
    goal_behavior='ignore'
)
        
# Make env
gpudrive_env = GPUDriveTorchEnv(
    config=env_config,
    data_loader = SceneDataLoader(
        root="data/processed/vbd",
        batch_size=1,
        dataset_size=4,
    ),
    render_config=RenderConfig(),
    max_cont_agents=32, # Maximum number of agents to control per scene
    device=device,
)

gpudrive_env.swap_data_batch()

In [3]:
# Action tensor to step through simulation
predicted_actions = torch.zeros((gpudrive_env.num_worlds, gpudrive_env.max_agent_count, gpudrive_env.episode_len, 10))

vbd_trajectories = VBDTrajectory.from_tensor(
    gpudrive_env.sim.vbd_trajectory_tensor(),
    device=device,
)

predicted_actions[:, :, :, 0] = vbd_trajectories.pos_x
predicted_actions[:, :, :, 1] = vbd_trajectories.pos_y
predicted_actions[:, :, :, 3] = vbd_trajectories.yaw
predicted_actions[:, :, :, 4] = vbd_trajectories.vel_x
predicted_actions[:, :, :, 5] = vbd_trajectories.vel_y

sim_state_frames = []
# fig = gpudrive_env.vis.plot_simulator_state(
#     time_steps=[0],
#     env_indices=[0],
#     zoom_radius=70,
# )[0]
# # Save initial state
# sim_state_frames.append(img_from_fig(fig))

for t in range(env_config.episode_len):
    # Step through simulation
    gpudrive_env.step_dynamics(predicted_actions[:, :, t, :])
    fig = gpudrive_env.vis.plot_simulator_state(
        time_steps=[t],
        env_indices=[0],
        zoom_radius=80,
    )[0]
    sim_state_frames.append(img_from_fig(fig))

In [4]:
mediapy.write_video('gpudrive/integrations/vbd/viz/train.gif', sim_state_frames, fps=20, codec="gif")